# Exercise 2: Feature Engineering - Data Encoding, Scaling, and Bias Mitigation

**Objective**: Explore the House Prices dataset, perform systematic feature engineering (encoding, transformation, scaling), and understand their impact on model behavior.

## Step 1: Environment Setup & Data Loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Load the House Prices dataset
# Note: Adjust the path if necessary based on your folder structure
file_path = "house-prices-advanced-regression-techniques/train.csv"
try:
    df = pd.read_csv(file_path)
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print(f"Error: File not found at {file_path}. Please check the path.")

# 2. Inspect the dataset structure
print("Shape:", df.shape)
df.info()
df.head()

## Step 2: Feature Type Separation (Numerical and Categorical)

In [ ]:
# 1. Separate features
numerical_features = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = df.select_dtypes(include=['object', 'category']).columns.tolist()

# 2. Create lists (already done above, but explicit printing)
print(f"Numerical Features ({len(numerical_features)}):", numerical_features)
print(f"Categorical Features ({len(categorical_features)}):", categorical_features)

# Justification:
# Separating features is crucial because they require different preprocessing steps.
# Numerical features typically need scaling and skewness correction.
# Categorical features need encoding (e.g., One-Hot, Ordinal) to be used by machine learning models.

## Step 3: Numerical Correlation Analysis

In [ ]:
# 1. Compute correlation matrix
corr_matrix = df[numerical_features].corr()

# 2. Visualize with heatmap
plt.figure(figsize=(15, 12))
sns.heatmap(corr_matrix, cmap='coolwarm', square=True)
plt.title("Correlation Heatmap of Numerical Features")
plt.show()

# 3. Focus on relationship with 'SalePrice'
print("Top 10 features correlated with SalePrice:\n", corr_matrix['SalePrice'].sort_values(ascending=False).head(10))

# Key Observations:
# 1. OverallQual has the strongest positive correlation with SalePrice.
# 2. GrLivArea (Living Area) is also highly correlated, indicating size matters.
# 3. GarageCars/GarageArea are correlated with each other (multicollinearity) and with Price.

## Step 4: Identify Top 10 Numerical Drivers

In [ ]:
# 1. Compute absolute correlation
abs_corr = df[numerical_features].corr()['SalePrice'].abs()

# 2. Exclude 'SalePrice'
abs_corr = abs_corr.drop('SalePrice')

# 3. Identify top 10
top_10_num_drivers = abs_corr.sort_values(ascending=False).head(10)

# Deliverable: Table
pd.DataFrame({'Feature': top_10_num_drivers.index, 'Abs_Correlation': top_10_num_drivers.values})

# Metric Justification: Absolute correlation is used because both strong positive and strong negative relationships are important predictors.

## Step 5: The "Hidden" Categorical Analysis

In [ ]:
# 1. Examine Top 10 numerical features
print("Top 10 Numerical Drivers:", top_10_num_drivers.index.tolist())

# 2. Identify hidden categorical feature: 'OverallQual' (Overall Quality) is often encoded as 1-10 but represents ordinal categories.
hidden_cat_feature = 'OverallQual'

# 3. Analysis
# a. Type: Ordinal (1=Very Poor, 10=Very Excellent)
# b. Plot against SalePrice
plt.figure(figsize=(10, 6))
sns.boxplot(x=hidden_cat_feature, y='SalePrice', data=df)
plt.title(f"{hidden_cat_feature} vs SalePrice")
plt.show()

# c. Comment on implicit encoding: The dataset uses integer encoding (1-10) which preserves the natural order. This is suitable for tree-based models and often linear models.

## Step 6: Inspect Pure Categorical Variables

In [ ]:
# 1. Examine remaining categorical columns
cat_cardinality = df[categorical_features].nunique().sort_values(ascending=False)

# 2. Check Cardinality
print("Cardinality of Categorical Features:")
print(cat_cardinality.head(10))

# Thinking about encoding:
# High Cardinality (e.g., Neighborhood) -> Target Encoding or Frequency Encoding might be better than One-Hot.
# Low Cardinality -> One-Hot Encoding is standard.

## Step 7: Random Forest Feature Importance

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

# Prepare data for RF (needs numbers)
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

# Simple preprocessing for Feature Importance check
# Identify cols again
num_cols = X.select_dtypes(include=[np.number]).columns
cat_cols = X.select_dtypes(include=['object']).columns

# Impute missing
num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='most_frequent')

X_num = pd.DataFrame(num_imputer.fit_transform(X[num_cols]), columns=num_cols)
X_cat = pd.DataFrame(cat_imputer.fit_transform(X[cat_cols]), columns=cat_cols)

# Ordinal Encode Categorical for RF
encoder = OrdinalEncoder()
X_cat_encoded = pd.DataFrame(encoder.fit_transform(X_cat), columns=cat_cols)

X_rf = pd.concat([X_num, X_cat_encoded], axis=1)

# 1. Train Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_rf, y)

# 2. Get top 10 features
importances = pd.Series(rf.feature_importances_, index=X_rf.columns).sort_values(ascending=False)
top_10_rf_features = importances.head(10)
print("Top 10 Random Forest Features:\n", top_10_rf_features)

# Plot Feature Importance
plt.figure(figsize=(10, 6))
top_10_rf_features.plot(kind='barh')
plt.title("Top 10 Feature Importances (Random Forest)")
plt.gca().invert_yaxis()
plt.show()

# Random Forest helps uncover non-linear effects because it uses decision trees that can capture complex interactions and non-monotonic relationships that linear correlation misses.

In [ ]:
# Plot top categorical features against SalePrice
top_cat_features = [f for f in top_10_rf_features.index if f in categorical_features]
print(f"\nTop Categorical Features from RF: {top_cat_features}")

# Plot up to 5 categorical features
for feat in top_cat_features[:min(5, len(top_cat_features))]:
    plt.figure(figsize=(12, 6))
    sns.boxplot(x=feat, y='SalePrice', data=df)
    plt.title(f"{feat} vs SalePrice (Random Forest Important Feature)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## Step 8: Encoding Strategy and Rationale

In [ ]:
# 1. Select top 5 categorical features (based on RF importance or domain knowledge from analysis)
top_cat_from_rf = [f for f in importances.index if f in categorical_features][:5]
if len(top_cat_from_rf) < 5:
    rest = [c for c in cat_cardinality.index if c not in top_cat_from_rf][:5-len(top_cat_from_rf)]
    top_cat_from_rf.extend(rest)
    
print("Selected Categorical Features for Encoding:", top_cat_from_rf)

# 2. Encoding Strategy for each feature
encoding_results = {}

for feat in top_cat_from_rf:
    print(f"\n--- Encoding {feat} ---")
    
    # Check cardinality
    n_unique = df[feat].nunique()
    print(f"Cardinality: {n_unique}")
    
    # Decision logic
    if n_unique > 10:
        print(f"Strategy: High cardinality → Frequency Encoding")
        # Frequency encoding
        freq_map = df[feat].value_counts(normalize=True).to_dict()
        encoding_results[feat] = df[feat].map(freq_map)
    else:
        print(f"Strategy: Low cardinality → One-Hot Encoding")
        # One-hot encoding
        encoding_results[feat] = pd.get_dummies(df[feat], prefix=feat)
    
    # Show sample
    print(f"Sample output:")
    if isinstance(encoding_results[feat], pd.DataFrame):
        display(encoding_results[feat].head())
    else:
        display(encoding_results[feat].head())

print("\nEncoding complete for all 5 features!")

## Step 9: Data Transformation and Scaling

In [ ]:
# Identify all high-skewness numerical features
print("=== Identifying High-Skewness Features ===")

# Calculate skewness for all numerical features (excluding target)
numerical_features_for_skew = [f for f in numerical_features if f != 'SalePrice']
skewness_data = df[numerical_features_for_skew].skew().sort_values(ascending=False)

# Define threshold (commonly |skewness| > 0.5 or 1.0)
skew_threshold = 0.75
high_skew_features = skewness_data[abs(skewness_data) > skew_threshold]

print(f"\nTotal numerical features: {len(numerical_features_for_skew)}")
print(f"Features with |skewness| > {skew_threshold}: {len(high_skew_features)}")
print(f"\nTop 10 most skewed features:")
print(high_skew_features.head(10))

# Create a summary table
skew_summary = pd.DataFrame({
    'Feature': high_skew_features.index,
    'Skewness': high_skew_features.values,
    'Abs_Skewness': abs(high_skew_features.values)
}).sort_values('Abs_Skewness', ascending=False)

print(f"\n=== All High-Skewness Features (|skew| > {skew_threshold}) ===")
display(skew_summary.head(15))

# Note the key features mentioned in PDF
pdf_features = ['SalePrice', 'GrLivArea', 'TotalBsmtSF']
print(f"\n=== Skewness of PDF-mentioned features ===")
for feat in pdf_features:
    if feat in df.columns:
        print(f"{feat}: {df[feat].skew():.3f}")

In [ ]:
from sklearn.preprocessing import PowerTransformer, StandardScaler, MinMaxScaler, RobustScaler

# 1. Find Skew
target_skew = df['SalePrice'].skew()
print(f"SalePrice Skew: {target_skew}")

# 2. Transformations
pt = PowerTransformer(method='yeo-johnson')
sp_log = np.log1p(df['SalePrice'])
sp_pt = pt.fit_transform(df[['SalePrice']])

# 3. Compare
print(f"Log Transform Skew: {sp_log.skew()}")
print(f"Yeo-Johnson Skew: {pd.Series(sp_pt.flatten()).skew()}")

plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1); sns.histplot(df['SalePrice'], kde=True); plt.title('Original')
plt.subplot(1, 3, 2); sns.histplot(sp_log, kde=True); plt.title('Log Transformed')
plt.subplot(1, 3, 3); sns.histplot(sp_pt, kde=True); plt.title('Yeo-Johnson')
plt.show()

# 4. Scaling Methods Comparison (using GrLivArea as example)
feature_to_scale = df[['GrLivArea']].dropna()

scalers = {
    'Standard': StandardScaler(),
    'MinMax': MinMaxScaler(),
    'Robust': RobustScaler()
}

plt.figure(figsize=(15, 4))
for i, (name, scaler) in enumerate(scalers.items()):
    scaled_data = scaler.fit_transform(feature_to_scale)
    plt.subplot(1, 3, i+1)
    sns.histplot(scaled_data, kde=True)
    plt.title(f"{name} Scaler")
plt.tight_layout()
plt.show()

# Recommendation: RobustScaler is often preferred if there are outliers (as seen in GrLivArea often). StandardScaler is standard for linear models if data is Gaussian-like.

In [ ]:
# Additional: Box-Cox Transformation (requires strictly positive values)
from sklearn.preprocessing import PowerTransformer

# Box-Cox for SalePrice (which is always positive)
pt_boxcox = PowerTransformer(method='box-cox')
sp_boxcox = pt_boxcox.fit_transform(df[['SalePrice']])

print(f"Box-Cox Skew: {pd.Series(sp_boxcox.flatten()).skew()}")

# Visualize all three transformations
plt.figure(figsize=(20, 5))
plt.subplot(1, 4, 1); sns.histplot(df['SalePrice'], kde=True); plt.title('Original')
plt.subplot(1, 4, 2); sns.histplot(sp_log, kde=True); plt.title('Log Transform')
plt.subplot(1, 4, 3); sns.histplot(sp_boxcox, kde=True); plt.title('Box-Cox Transform')
plt.subplot(1, 4, 4); sns.histplot(sp_pt, kde=True); plt.title('Yeo-Johnson Transform')
plt.tight_layout()
plt.show()

In [ ]:
# Analyze TotalBsmtSF (as requested in PDF)
if 'TotalBsmtSF' in df.columns:
    print("\n=== TotalBsmtSF Analysis ===")
    totalbsmt_skew = df['TotalBsmtSF'].skew()
    print(f"TotalBsmtSF Original Skew: {totalbsmt_skew}")
    
    # Apply transformations
    # Handle zeros by adding 1 before log
    tbsf_log = np.log1p(df['TotalBsmtSF'])
    print(f"After Log Transform: {tbsf_log.skew()}")
    
    # Yeo-Johnson (handles zeros)
    pt_yj = PowerTransformer(method='yeo-johnson')
    tbsf_yj = pt_yj.fit_transform(df[['TotalBsmtSF']])
    print(f"After Yeo-Johnson: {pd.Series(tbsf_yj.flatten()).skew()}")
    
    # Visualize
    plt.figure(figsize=(15, 4))
    plt.subplot(1, 3, 1); sns.histplot(df['TotalBsmtSF'], kde=True); plt.title(f'TotalBsmtSF Original\nSkew: {totalbsmt_skew:.2f}')
    plt.subplot(1, 3, 2); sns.histplot(tbsf_log, kde=True); plt.title(f'Log Transform\nSkew: {tbsf_log.skew():.2f}')
    plt.subplot(1, 3, 3); sns.histplot(tbsf_yj, kde=True); plt.title(f'Yeo-Johnson\nSkew: {pd.Series(tbsf_yj.flatten()).skew():.2f}')
    plt.tight_layout()
    plt.show()
else:
    print("TotalBsmtSF not found in dataset")

In [ ]:
# Add MaxAbsScaler to comparison
from sklearn.preprocessing import MaxAbsScaler

feature_to_scale = df[['GrLivArea']].dropna()

scalers_complete = {
    'Standard': StandardScaler(),
    'MinMax': MinMaxScaler(),
    'Robust': RobustScaler(),
    'MaxAbs': MaxAbsScaler()
}

plt.figure(figsize=(20, 4))
for i, (name, scaler) in enumerate(scalers_complete.items()):
    scaled_data = scaler.fit_transform(feature_to_scale)
    plt.subplot(1, 4, i+1)
    sns.histplot(scaled_data, kde=True)
    plt.title(f"{name} Scaler")
plt.tight_layout()
plt.show()

print("\nScaler Recommendations:")
print("- StandardScaler: Best for normally distributed data without outliers")
print("- MinMaxScaler: When you need a specific range [0,1], but sensitive to outliers")
print("- RobustScaler: When data has outliers (uses median and IQR)")
print("- MaxAbsScaler: When you need range [-1,1] and want to preserve sparsity")

In [ ]:
# Create comprehensive skewness comparison table
skew_comparison = pd.DataFrame({
    'Feature': ['SalePrice', 'SalePrice', 'SalePrice', 'TotalBsmtSF', 'TotalBsmtSF'],
    'Transformation': ['Original', 'Log', 'Yeo-Johnson', 'Original', 'Log'],
    'Skewness': [
        df['SalePrice'].skew(),
        sp_log.skew(),
        pd.Series(sp_pt.flatten()).skew(),
        df['TotalBsmtSF'].skew() if 'TotalBsmtSF' in df.columns else None,
        tbsf_log.skew() if 'TotalBsmtSF' in df.columns else None
    ]
})

print("\n=== Skewness Comparison Table ===")
display(skew_comparison)
print("\nConclusion: Power transformations (Log, Box-Cox, Yeo-Johnson) effectively reduce skewness close to 0.")

## Step 10: Model Comparison

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Data Prep - Drop rows with missing target
df_clean = df.dropna(subset=['SalePrice'])
y = df_clean['SalePrice']
X = df_clean.drop('SalePrice', axis=1)

# Select only numerical cols for baseline to simplify
X_baseline = X.select_dtypes(include=[np.number]).fillna(0) # Simple fill

# Split
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X_baseline, y, test_size=0.2, random_state=42)

# 1. Baseline Model
lr_base = LinearRegression()
lr_base.fit(X_train_b, y_train_b)
y_pred_b = lr_base.predict(X_test_b)
rmse_b = np.sqrt(mean_squared_error(y_test_b, y_pred_b))
r2_b = r2_score(y_test_b, y_pred_b)

# 2. Improved Model (Log Target + Scaled Features)
# Log Transform Target
y_train_imp = np.log1p(y_train_b)
y_test_imp = np.log1p(y_test_b)

# Scale Features
scaler = StandardScaler()
X_train_imp = scaler.fit_transform(X_train_b)
X_test_imp = scaler.transform(X_test_b)

lr_imp = LinearRegression()
lr_imp.fit(X_train_imp, y_train_imp)
y_pred_imp_log = lr_imp.predict(X_test_imp)
y_pred_imp = np.expm1(y_pred_imp_log) # Inverse log

rmse_imp = np.sqrt(mean_squared_error(y_test_b, y_pred_imp))
r2_imp = r2_score(y_test_b, y_pred_imp)

# 4. Compare
comparison = pd.DataFrame({
    'Model': ['Baseline', 'Improved (Log+Scaled)'],
    'RMSE': [rmse_b, rmse_imp],
    'R2': [r2_b, r2_imp]
})
print(comparison)

# Commentary:
# Linear models typically perform better when the target variable is normally distributed (skewness corrected) and features are on the same scale.
# Log transformation of SalePrice is highly effective because prices are usually log-normally distributed.